# COVID Dashboard

This dashboard gathers data from JOhn Hopkins, Apple, Google, and the Institute for Health Metrics and Evaluation (IMHE) at the University of Washington allowing a collected view of all three datasets.  It was constructed as the Software Engineering (CSIS 340) Final Project for Dio Lopez Vasquez, Juan Cabanela, and Luke Hebert in Summer 2020.

In [1]:
# This forces a reload of any external library file if it changes.  
# Useful when developing external libraries since otherwise Jupyter 
# will not re-import any library without restarting the python kernel.

%load_ext autoreload
%autoreload 2

In [2]:
# Enable ipywidget backend for matplotlib
%matplotlib widget

In [3]:
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from ipywidgets import Layout
from ipywidgets import widgets

# Import COVID IO routines from external python libraries
import COVIDlib.data_IO as COVID_IO
import COVIDlib.dashboard_IO as COVID_Dash

## Define variables of interest below
data_dir = 'our_data/'    # Data directory for the COVID datafiles

## Set defaults for menus here
state1 = "Minnesota"
county1 = "Clay"  # Must be county in state1
state2 = "North Dakota"
county2 = "Cass"  # Must be county in state2

In [4]:
# Load all the dataframes into memory
#print("Loading and Preprocessing COVID Dataset ... ", end='')

start= time.perf_counter()
# Retrieve John Hopkins dataframes and add "rates" of deaths/infections
(JH_state_df, JH_cnty_df) = COVID_IO.PtoCDRDataFrames()
JH_state_df = COVID_Dash.cleanJHdata(JH_state_df)
JH_cnty_df = COVID_Dash.cleanJHdata(JH_cnty_df)

# Construct dictionary of FIPS values by placename
FIPSd = COVID_Dash.build_fipsdict(JH_cnty_df, JH_state_df)
# Build a list of all states to put in menus and then dictionary of lists of counties for each state
StatesList = COVID_Dash.BuildStatesList()
CountiesDict = COVID_Dash.BuildCountiesList(JH_cnty_df, StatesList)
# Update FIPS numbers for the states and countys based on defaults
FIPSstate1 = COVID_Dash.GetFIPS(FIPSd, state1)
FIPScnty1 = COVID_Dash.GetFIPS(FIPSd, state1, county1)
FIPSstate2 = COVID_Dash.GetFIPS(FIPSd, state2)
FIPScnty2 = COVID_Dash.GetFIPS(FIPSd, state2, county2)
# Load variable descriptions
JHVarDict = COVID_Dash.BuildJHVarDict()

# Retrieve Apple Mobility Dataframes
(aapl_cnty_df, aapl_state_df) = COVID_IO.PtoAAPLMobilityDataFrames()
COVID_Dash.cleanAAPLdata(aapl_cnty_df)
COVID_Dash.cleanAAPLdata(aapl_state_df)
# Retrieve Google Mobility Dataframes
(goog_cnty_df, goog_state_df) = COVID_IO.PtoGOOGMobilityDataFrames()
# Load mobility variable descriptions
mobilityVarDict = COVID_Dash.BuildMobilityVarDict()

# Retrieve IMHE Dataframes
(summary_df, hospitalization_df) = COVID_IO.PtoIMHEDataFrames()

end= time.perf_counter()

#print(f"Done ({end-start:0.2f} sec)")
# print(FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2)

In [24]:
##
## Functions to call to respond to Widget changes
##

def county_changed(listnum, value):
    # This function updates FIPS values that need to be changed when the county value changes
    global FIPSd, FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2
    global state1_drop, state2_drop
    
    # Update appropriate FIPS records
    if (listnum == 1):
        FIPSstate1 = COVID_Dash.GetFIPS(FIPSd, state1_drop.value)
        FIPScnty1 = COVID_Dash.GetFIPS(FIPSd, state1_drop.value, value)
    else:
        FIPSstate2 = COVID_Dash.GetFIPS(FIPSd, state2_drop.value)
        FIPScnty2 = COVID_Dash.GetFIPS(FIPSd, state2_drop.value, value)
    
    # Trigger update of textual information
    refresh_text()
    
    # Trigger the plots to redraw here
    refresh_plot1()
    
    return


def county1_changed(change):
    # Update the county 1 value
    county_changed(1, change.new)
    return


def county2_changed(change):
    # Update the county 2 value
    county_changed(2, change.new)
    return


def state_changed(listnum, value):
    # This function updates the appropriate counties list widget
    global county1_drop, county2_drop, CountiesDict
    global state1, state2, county1, county2
    
    # Define dictionary of widgets
    widgets_dict = {1: county1_drop, 2: county2_drop}
    cnty_wdgt = widgets_dict[listnum]
    
    # Update the Corresponding County Widget
    cnty_wdgt.options=CountiesDict[value]
    cnty_wdgt.value=CountiesDict[value][0]
    
    # This change in the county values should trigger appropriate county_changed function    
    return
    
    
def state1_changed(change):
    # Update the state 1 value
    state_changed(1, change.new)
    return
    
    
def state2_changed(change):
    # Update the state 2 value
    state_changed(2, change.new)
    return


def Var1_changed(change):
    # This function handles changes to the time series 1 plot variable
    global FIPSd, FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2
        
    # Trigger the plot redraw here
    refresh_plot1()
    return


def refresh_plot1():
    # This redraws the plot using the John Hopkins data
    global JH_state_df, JH_cnty_df, JHVarDict, Var1_drop, output1
    global FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2, fig1, ax1
    
    # Collect states and counties to plot
    states = [FIPSstate1, FIPSstate2]
    cntys  = [FIPScnty1, FIPScnty2]
    
    # Clear previous plot (NOT WORKING)
    fig1.clear
    
    # Plot state data
    COVID_Dash.ts_plot(JH_state_df, Var1_drop.value, states, fig=fig1, ax=ax1) 

    # Plot county data
    if not JHVarDict[Var1_drop.value]['stateonly']:
        COVID_Dash.ts_plot(JH_cnty_df, Var1_drop.value, cntys, fig=fig1, ax=ax1) 


def refresh_text():
    global JH_state_df, JH_cnty_df, FIPSstate1, FIPScnty1, FIPSstate2, FIPScnty2
    
    # Get textual status
    html_state1 = COVID_Dash.html_status(JH_state_df, FIPSstate1, summary_df, BedsStatus=False, Display=False)
    html_cnty1 = COVID_Dash.html_status(JH_cnty_df, FIPScnty1, summary_df, BedsStatus=False, Display=False)
    html_state2 = COVID_Dash.html_status(JH_state_df, FIPSstate2, summary_df, BedsStatus=False, Display=False)
    html_cnty2 = COVID_Dash.html_status(JH_cnty_df, FIPScnty2, summary_df, BedsStatus=False, Display=False)
    html_out = html_state1 + html_cnty1 + html_state2 + html_cnty2
    
    # Update text field
    text_display.value=html_out

In [25]:
##
## Build Widgets to Display
##

##
## LOCATION CONTROL WIDGETS
##
# Build Dropdown Widgets for first and second county/state combinations with hard-coded defaults for Clay and Cass County in Minnesota
loc1_label = widgets.HTML(value='<b style="font-size:120%">Location #1:</b> ')
loc2_label = widgets.HTML(value='<b style="font-size:120%">Location #2:</b> ')
spacer = widgets.HTML(value='&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;', layout=widgets.Layout(width='50px') )
state1_drop = widgets.Dropdown(options=StatesList,
                               value=state1,
                               description='State #1:',
                               disabled=False, )
county1_drop = widgets.Dropdown(options=CountiesDict[state1],
                               value=county1,
                               description='County #1:',
                               disabled=False, )
state2_drop = widgets.Dropdown(options=StatesList,
                               value=state2,
                               description='State #2:',
                               disabled=False, )
county2_drop = widgets.Dropdown(options=CountiesDict[state2],
                               value=county2,
                               description='County #2:',
                               disabled=False, )
# Reset FIPS values
FIPSstate1 = COVID_Dash.GetFIPS(FIPSd, state1_drop.value)
FIPScnty1 = COVID_Dash.GetFIPS(FIPSd, state1_drop.value, county1_drop.value)
FIPSstate2 = COVID_Dash.GetFIPS(FIPSd, state2_drop.value)
FIPScnty2 = COVID_Dash.GetFIPS(FIPSd, state2_drop.value, county2_drop.value)
        
# CONNECTIONS TO FUNCTIONS TO HANDLE LOCATION CHANGES
state1_drop.observe(state1_changed, 'value')
state2_drop.observe(state2_changed, 'value')
county1_drop.observe(county1_changed, 'value')
county2_drop.observe(county2_changed, 'value')

##
## TEXT INFORMATION DISPLAY
##
text_display = widgets.HTML(value='Text Information Field',
                            layout=widgets.Layout(width='500px'))
refresh_text() # Update text field with current information

##
## PLOT OF JOHN HOPKINS DATA
##
JHvars_menu = []
JHvars = []
for key in JHVarDict:
    # Build variable name list for Dropdown
    JHvars_menu.append((JHVarDict[key]['descript'], key))
    JHvars.append(key)
    
# This selects the John Hopkins Time Series Data to show in Display 1
style = {'description_width': 'initial'}
Var1_drop = widgets.Dropdown(options=JHvars_menu,
                               value=JHvars[0],
                               description='Time Series 1:',
                               disabled=False, 
                               style=style)
Var1_drop.observe(Var1_changed, 'value')

# Create a Matplotlib subplot within output
output1 = widgets.Output()
with output1:
        fig1, ax1 = plt.subplots(constrained_layout=True, figsize=(5, 5))   
        fig1.canvas.toolbar_visible = False
        fig1.canvas.header_visible = False
        fig1.canvas.footer_visible = False
        fig1.canvas.resizable = False
        fig1.canvas.capture_scroll = False

refresh_plot1() # Update this plot

# CONNECTIONS TO FUNCTIONS TO HANDLE TIME SERIES 1 CHANGES



In [26]:
## BUILD FULL INTERFACE:
## Build the display from widgets using a series of HBox (Horizontal boxes) and VBox (Vertical boxes)
## to position the widgets relative to each other.

# LOCATION CONTROLS
loc1_controls = widgets.VBox([state1_drop, county1_drop], 
                             layout=widgets.Layout(align_content='center', align_items='center'))
loc2_controls = widgets.VBox([state2_drop, county2_drop], 
                             layout=widgets.Layout(align_content='center', align_items='center'))
row1 = widgets.HBox([loc1_label, loc1_controls, spacer, loc2_label, loc2_controls])

# BUILD WIDGET FOR TEXTBOX/PLOTS IN TOP ROW
JHplotContainer  = widgets.VBox([Var1_drop, output1], 
                             layout=widgets.Layout(align_content='center', align_items='center',
                                                   width='700px',overflow_x='hidden', overflow_y='hidden'))
row2 = widgets.HBox([text_display, JHplotContainer], layout=widgets.Layout(border='solid'))

# BUILD WIDGET FOR PLOTS IN MIDDLE ROW

# BUILD WIDGET FOR MAP FOR BOTTOM ROW

MainDisplay = widgets.VBox([row1, row2], 
                           layout=widgets.Layout(align_content='center', align_items='center',
                                                 margin='0px', width='1200px', 
                                                 overflow_x='hidden', overflow_y='hidden'))
Main = display(MainDisplay)


# OLD EXAMPLE CODE (DELETE EVENTUALLY)
#incl_slider = widgets.IntSlider(min=0, max=80, step=1, value=incl,
#                                 continuous_update=False, orientation='horizontal',
#                                 readout=True, readout_format='04d',
#                                 layout=widgets.Layout(height='75px', min_height='50px', max_height='100px',
#                                                       min_width='50px', width='200px', max_width='300px',
#                                                       overflow_x='hidden', overflow_y='hidden'))
#imgfile_label = widgets.HTML(value='Image Filename: ', 
#                             layout=widgets.Layout(min_width='70px', width='100px', max_width='200px'))
#imgfile_name = widgets.Text(value=image_filename, 
#                             layout=widgets.Layout(min_width='100px', width='200px', max_width='300px'))
